In [55]:
import json
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc

In [142]:
class ProfileThresholdHate():
    def __init__(self):
        with open("data/actual/twitter_following_385.json","r") as f:
            data=json.load(f)
        assert len(data.keys())==385
        self.g=nx.DiGraph()
        for each_user in data:
            followers=data[each_user]
            if len(followers)==0:
                self.g.add_node(int(each_user))
            else:
                for each_follower in followers:
                    self.g.add_edge(int(each_user), int(each_follower))
        assert self.g.number_of_nodes()==385
        self.model = ep.ProfileThresholdModel(g)
        
    def config_new_model(self, infected_nodes):
        self.config = mc.Configuration()
        self.config.add_model_parameter('blocked', 0)
        self.config.add_model_parameter('adopter_rate', 0)
        if not infected_nodes:
            infected_nodes = [94152234]
        self.config.add_model_initial_configuration("Infected", infected_nodes)
        self.threshold = 0.75
        self.profile = 0.85
        for i in self.g.nodes():
            self.config.add_node_configuration("threshold", i, threshold)
            self.config.add_node_configuration("profile", i, profile)
        self.model.set_initial_status(self.config)
        
    def run_model(self, infected_nodes):
        self.config_new_model(infected_nodes)
        iterations = self.model.iteration_bunch(2)
#         print(set(iterations[1]['status'].keys()))
        return set(iterations[1]['status'].keys())
    
    def run_model_timewise(self, infected_nodes, max_iterations=50):
        infected_updated=set()
        for each_infected in infected_nodes:
            infected_updated.add(each_infected)
            for each_follower in self.g[each_infected]:
                infected_updated.add(each_follower)
        infected_nodes = list(infected_updated)
        self.config_new_model(infected_nodes)
        iterations = self.model.iteration_bunch(max_iterations)
        return iterations

In [147]:
pth=ProfileThresholdHate()
# update_nodes=pth.run_model([])

In [166]:
import requests
json_={}
json_["infected_node"]=["94152234"] # put a different user_id here
print(json.dumps(json_))
r = requests.post(url="http://localhost:8092/api/predict_single_iter", data=json.dumps(json_),  headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
print(r.text)

{"infected_node": ["94152234"]}
{
  "1008609815373885440": {
    "followers": 23, 
    "following": 104, 
    "location": "Dani Limbada, Ahmadabad City", 
    "score": 0
  }, 
  "10196452": {
    "followers": 14160, 
    "following": 14127, 
    "location": "Kent, England", 
    "score": 0
  }, 
  "1022475648001101825": {
    "followers": 9, 
    "following": 127, 
    "location": "United States", 
    "score": 0
  }, 
  "1022946567173615617": {
    "followers": 7831, 
    "following": 8044, 
    "location": "United States", 
    "score": 0
  }, 
  "1023985966338334720": {
    "followers": 114, 
    "following": 142, 
    "location": "Mumbai india", 
    "score": 0
  }, 
  "1025642153362575360": {
    "followers": 21, 
    "following": 209, 
    "location": "Calcutta ", 
    "score": 0
  }, 
  "1025922902414188544": {
    "followers": 7396, 
    "following": 7669, 
    "location": "Anytown USA", 
    "score": 0
  }, 
  "102653259": {
    "followers": 814, 
    "following": 1146, 
    "

In [160]:
update_nodes

{114273855,
 207435031,
 296804016,
 3110077936,
 779678725663289344,
 863377035661647877,
 898161074284965888,
 904074485443256320,
 906082234490290176,
 949726094919458817,
 1101285503582781441}

In [149]:
pth.run_model_timewise([94152234])

100%|██████████| 50/50 [00:00<00:00, 980.73it/s]


[{'iteration': 0,
  'status': {432474709: 0,
   589089799: 0,
   94152234: 1,
   49965183: 0,
   91138156: 0,
   21059255: 0,
   207435031: 1,
   380648579: 0,
   745833618874970112: 0,
   759511561648140288: 0,
   231724292: 0,
   1128914551792181248: 0,
   898161074284965888: 1,
   10196452: 0,
   196599046: 0,
   805646428274098176: 0,
   16942658: 0,
   831150536183398400: 0,
   961824768356794368: 0,
   817802951435685888: 0,
   1022475648001101825: 0,
   356820490: 0,
   127643327: 0,
   3180616607: 0,
   742375062477492228: 0,
   1193782281984335873: 0,
   883702682808197121: 0,
   194022170: 0,
   897499446: 0,
   711312535048347648: 0,
   19080151: 0,
   19980832: 0,
   4834659851: 0,
   20612711: 0,
   854449144336982017: 0,
   30197309: 0,
   793332751378227200: 0,
   152630167: 0,
   1167406031233667073: 0,
   1232108631253905409: 0,
   941438450368053248: 0,
   3164628703: 0,
   441817658: 0,
   82815038: 0,
   83319968: 0,
   819961702431092736: 0,
   73043514: 0,
   2314

In [131]:
from copy import deepcopy
class FormOutput():
    def __init__(self):
        with open("data/actual/twitter_following_385.json","r") as f:
            data=json.load(f)
        assert len(data.keys())==385
        self.g=nx.DiGraph()
        for each_user in data:
            followers=data[each_user]
            if len(followers)==0:
                self.g.add_node(int(each_user))
            else:
                for each_follower in followers:
                    self.g.add_edge(int(each_user), int(each_follower))
        with open("data/actual/twitter_profile_385.json","r") as f:
            self.user_data=json.load(f)
    def update_iter_1_scores(self, update_nodes):
        return_scores = deepcopy(self.user_data)
        for each_user in return_scores:
            if each_user in update_nodes:
                return_scores[each_user]['score']=1
            else:
                return_scores[each_user]['score']=0
        return return_scores

In [132]:
out_=FormOutput()
out_.update_iter_1_scores(update_nodes)

{'432474709': {'following': 1633,
  'followers': 1155,
  'location': 'London',
  'score': 0},
 '745833618874970112': {'following': 946,
  'followers': 747,
  'location': 'Colorado, USA',
  'score': 0},
 '231724292': {'following': 287, 'followers': 96, 'location': '', 'score': 0},
 '805646428274098176': {'following': 3097,
  'followers': 270,
  'location': 'Indianapolis, IN',
  'score': 0},
 '356820490': {'following': 3648,
  'followers': 16872,
  'location': '',
  'score': 0},
 '897499446': {'following': 4563,
  'followers': 4386,
  'location': 'Gitchi Gamii, USA',
  'score': 0},
 '711312535048347648': {'following': 1831,
  'followers': 817,
  'location': 'Portugal',
  'score': 0},
 '724440955147808768': {'following': 744,
  'followers': 351,
  'location': '',
  'score': 0},
 '1198796864021073920': {'following': 175,
  'followers': 29,
  'location': 'India',
  'score': 0},
 '1214601012981510145': {'following': 594,
  'followers': 155,
  'location': '',
  'score': 0},
 '1093476339896598

In [237]:
with open("data/actual/twitter_profile_385.json","r") as f:
    data2=json.load(f)
data2["1217043540208783362"]["followers"]
# follower_followee_ratio = {}
# max_=0
# for each_user in data:
#     ratio = data[each_user]["followers"]/data[each_user]["following"]
#     follower_followee_ratio[each_user] = ratio
#     if ratio>max_:
#         max_=ratio
# for each in follower_followee_ratio:
#     follower_followee_ratio[each]=follower_followee_ratio[each]/max_
# with open("data/results/follower_followee_max_norm_ratio385.json","w") as f:
#     json.dump(follower_followee_ratio, f, indent=True)

1581

In [196]:
with open("data/actual/twitter_timeline_385.json","r") as f:
    data=json.load(f)


[]

In [224]:
tweets = data["1217043540208783362"][0]["full_text"]
rc=data["1217043540208783362"][0]["retweet_count"]

In [225]:
# tweets
json_ ={'input_tweet':tweets, "model":"D"}
# json_
print(json.dumps(json_))

{"input_tweet": "RT @Azad_India20: #\u0930\u093e\u092e\u0926\u0947\u0935_\u092c\u093e\u092c\u093e_\u092a\u0948\u0938\u0947_\u0928\u093f\u0915\u093e\u0932\u094b\n\u0930\u093e\u092e\u0926\u0947\u0935 \u092c\u093e\u092c\u093e \u0926\u0947\u0936 \u0915\u0947 10 \u0938\u092c\u0938\u0947 \u0905\u092e\u0940\u0930 \u0932\u094b\u0917\u094b\u0902 \u092e\u0947\u0902 \u0936\u0941\u092e\u093e\u0930 Businesman \u0939\u0948, \u092f\u0947 \u092c\u093e\u0924 \u0905\u0932\u0917 \u0939\u0948 \u0915\u0940 \u0935\u094b \u092a\u0948\u0938\u094b\u0902 \u0915\u0947 \u0932\u093f\u090f\u2026", "model": "D"}


In [232]:
'''
Json format for hate detection api:
[
{'input_tweet': 'tweet text'},
{'input_tweet': 'tweet text'},...
]
Response from api:
[
{'input_tweet': 'tweet text',
'pred_class': 0/1/2,
'pred_label': 'hate speech'/'offensive language'/'neither'},...
]
'''
# json_ = [{'input_tweet':t.text} for t in tweets]
# print(json.dumps(json_))
r = requests.post(url="http://0.0.0.0:8090/api/predict_single", data=json.dumps(json_),  headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
# print(r.text)
json.loads(r.text)["pred_class"]
# labels = [t['pred_class']  for t in json.loads(r.text)]
# return labels

2

In [240]:
hate_nonhate_ratio={}
# retweet_counts={}
count=1
for each_user in data:
    print(count)
    count+=1
    hate_spread_score=0
    non_hate_spread_score = 0
    followers = data2[each_user]["followers"]
    for each_tweet in data[each_user]:
        tweet_data = each_tweet["full_text"]
        retweet_count = each_tweet["retweet_count"]
        retweet_count += 1
        json_ = {'input_tweet':tweet_data, "model":"D"}
        r = requests.post(url="http://0.0.0.0:8090/api/predict_single", data=json.dumps(json_),  headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
        label = json.loads(r.text)["pred_class"]
        if label==0 or label==1:
            hate_spread_score+=retweet_count/followers
        else:
            non_hate_spread_score+=retweet_count/followers
    if hate_spread_score and non_hate_spread_score:
        ratio = hate_spread_score/non_hate_spread_score
    else:
        ratio=0
    hate_nonhate_ratio[each_user]=ratio

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [241]:
len(hate_nonhate_ratio)

382

In [242]:
len(data)

382

In [247]:
no_data=set(follower_followee_ratio.keys()).difference(set(hate_nonhate_ratio.keys()))
no_data

{'28997944', '853241196910698496', '91254133'}

In [248]:
for uid in no_data:
    hate_nonhate_ratio[uid]=0

In [250]:
# hate_nonhate_ratio

In [251]:
with open("data/results/hate_nonhate_ration385.json","w") as f:
    json.dump(hate_nonhate_ratio,f,indent=True)